
Important Libraries

In [ ]:
# Importerar nödvändiga Libraries

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
import os

# Importing Deep Learning Libraries ipykernel

from keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Input, Dropout, GlobalAveragePooling2D, Flatten, Conv2D, BatchNormalization, Activation, MaxPooling2D
from keras.models import Model, Sequential
from keras.optimizers import Adam, SGD, RMSprop


Displaying Images

In [ ]:

folder_path = 'D:/DataSets/images/images/'
picture_size = 48

In [ ]:
### Visualiserar exempelbilder för att kontrollera att bilderna laddas in korrekt och ser ut som förväntat


expression = 'disgust'

plt.figure(figsize=(12, 12))
for i in range(1, 10, 1):
  plt.subplot(3, 3, i)
  img = load_img(folder_path+'train/'+expression+'/'+
                 os.listdir(folder_path + 'train/' + expression)[i], target_size=(picture_size, picture_size))
  plt.imshow(img)
plt.show

In [ ]:
# Förbereder tränings- och valideringsdata för att användas i modellen

batch_size = 128

datagen_train = ImageDataGenerator()
datagen_val = ImageDataGenerator()

train_set = datagen_train.flow_from_directory(folder_path+'train',
                                              target_size = (picture_size, picture_size),
                                              color_mode ='grayscale',
                                              batch_size = batch_size,
                                              class_mode = 'categorical',
                                              shuffle = True)

test_set = datagen_val.flow_from_directory(folder_path+'validation',
                                              target_size = (picture_size, picture_size),
                                              color_mode = 'grayscale',
                                              batch_size = batch_size,
                                              class_mode = 'categorical',
                                              shuffle = False)


In [ ]:
train_set[0]

Model Building

In [ ]:
# Skapar modellen med olika lager för att extrahera funktioner och klassificera bilder i 7 kategorier

no_of_classes = 6

model = Sequential()

# Första konvolutionslagret för att extrahera lågnivåfunktioner
model.add(Conv2D(64, (3,3), padding = 'same', input_shape = (48, 48, 1)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.25))

# Andra konvolutionslagret för att extrahera mer komplexa funktioner
model.add(Conv2D(128, (5,5), padding = 'same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.25))

# Tredje konvolutionslagret för att extrahera ännu mer komplexa funktioner
model.add(Conv2D(512, (3,3), padding = 'same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.25))

# Fjärde konvolutionslagret för att ytterligare förfina funktionerna
model.add(Conv2D(128, (5,5), padding = 'same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.25))

# Flatten lagret för att konvertera 2D-data till 1D-data
model.add(Flatten())

# Första fullt anslutna lagret för att kombinera extraherade funktioner
model.add(Dense(256))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

# Andra fullt anslutna lagret för att ytterligare kombinera funktioner
model.add(Dense(256))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

# Utgångslager med softmax-aktivering för att klassificera i 6 kategorier
model.add(Dense(no_of_classes, activation= 'softmax'))

# Kompilering av modellen med Adam-optimizer och kategorisk korsentropi som förlustfunktion
opt = Adam(learning_rate = 0.0001)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Fitting the model with Training and Validation Data

In [ ]:
# Sparar modellen så att den kan användas senare, samt tillämpar olika regulariseringsmetoder för att förbättra modellen

from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

# Checkpoint för att spara den bästa modellen baserat på valideringsnoggrannhet
checkpoint = ModelCheckpoint("./face.tf", monitor="val_accuracy", verbose=1, save_best_only=True, mode='max')

# Early stopping för att avbryta träningen när noggrannheten slutar förbättras för att undvika överträning
early_stopping = EarlyStopping(monitor='accuracy',
                               min_delta=0,
                               patience=3,
                               verbose=1,
                               restore_best_weights=True)

# Reduce learning rate on plateau för att minska inlärningshastigheten när noggrannheten slutar förbättras
reduce_learningrate = ReduceLROnPlateau(monitor='accuracy',
                                        factor=0.2,
                                        patience=3,
                                        verbose=1,
                                        min_delta=0.0001)

callbacks_list = [early_stopping, checkpoint, reduce_learningrate]

epochs = 48

In [ ]:
# Tränar modellen med träningsdata och validerar med valideringsdata
# Använder callbacks för att spara bästa modell, avbryta vid överträning och justera inlärningshastigheten

history = model.fit(          
    train_set,
    steps_per_epoch= train_set.n//train_set.batch_size,
    epochs=epochs, 
    validation_data = test_set,
    validation_steps=  test_set.n//test_set.batch_size,
    callbacks=callbacks_list
    )

Plotting Accuracy & Loss

In [ ]:
## Visualiserar tränings- och valideringsförlust samt noggrannhet för att bedöma modellens prestanda och avgöra om ytterligare träning behövs

plt.style.use('dark_background')

plt.figure(figsize=(20, 10))

# Plot för tränings- och valideringsförlust
plt.subplot(1, 2, 1)
plt.title('Optimizer : Adam', fontsize=10)
plt.ylabel('Loss', fontsize=16)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend(loc='upper right')

# Plot för tränings- och valideringsnoggrannhet
plt.subplot(1, 2, 2)
plt.ylabel('Accuracy', fontsize=16)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend(loc='lower right')

plt.show()